In [1]:
import os
os.chdir(r'/home/ubuntu/congressional-stock-trading-monitor/')

In [2]:
from cstm.database_helpers import get_db_connection, table_name
from cstm.query import *

In [3]:
database_path = r"database/database.db"

In [4]:
cur = get_db_connection(database_path).cursor()

In [5]:
temp_query = f"SELECT company, ticker, id, member_name, transaction_date, " \
             f"CASE transaction_type WHEN 'P' THEN value_lb ELSE 0 END AS purchase_lb, "\
             f"CASE transaction_type WHEN 'P' THEN value_ub ELSE 0 END AS purchase_ub, "\
             f"CASE transaction_type WHEN 'S' THEN value_lb ELSE 0 END AS sale_lb, "\
             f"CASE transaction_type WHEN 'S' THEN value_ub ELSE 0 END AS sale_ub "\
             f"FROM {table_name} "
full_query = f"WITH temp as ({temp_query}) "\
             f"SELECT "\
             f"COUNT(id) AS num_transactions, "\
             f"COUNT(DISTINCT member_name) AS num_members, "\
             f"SUM(purchase_lb) as purchase_lb, "\
             f"SUM(purchase_ub) as purchase_ub, "\
             f"SUM(sale_lb) as sale_lb, "\
             f"SUM(sale_ub) as sale_ub, "\
             f"strftime(\'%d-%m-%Y\', transaction_date) as transaction_date "\
             f"FROM temp " \
             f"WHERE transaction_date < \'2022-01-01\'" \
             f"GROUP BY transaction_date"

In [8]:
from visualization.helper_functions import purchase_sale_sum_on_time
from datetime import datetime

date_upper = datetime.now()
date_lower = datetime(2013,1,1)
full_query = purchase_sale_sum_on_time(date_lower=date_lower, date_upper=date_upper, ticker=None,
                                       company_name=None, member_name=None)

print(full_query)


WITH temp as (SELECT company, ticker, id, member_name, transaction_date, CASE transaction_type WHEN 'P' THEN value_lb ELSE 0 END AS purchase_lb, CASE transaction_type WHEN 'P' THEN value_ub ELSE 0 END AS purchase_ub, CASE transaction_type WHEN 'S' THEN value_lb ELSE 0 END AS sale_lb, CASE transaction_type WHEN 'S' THEN value_ub ELSE 0 END AS sale_ub FROM all_transaction)SELECT COUNT(id) AS num_transactions, COUNT(DISTINCT member_name) AS num_members, SUM(purchase_lb) as purchase_lb, SUM(purchase_ub) as purchase_ub, SUM(sale_lb) as sale_lb, SUM(sale_ub) as sale_ub, strftime('%d-%m-%Y', transaction_date) as transaction_date  FROM temp WHERE (transaction_date BETWEEN '01-01-2013' AND '06-05-2022' AND ticker = 'None' AND member_name = 'None' AND company_name = 'None') GROUP BY transaction_date


In [7]:
data = cur.execute(full_query)
print(type(data))
column_names = [col[0] for col in data.description]
data = data.fetchall()

OperationalError: no such column: company_name

In [ ]:
print(type(data[0]))

In [ ]:
from datetime import datetime


def convert_date_string_lst_to_datetime(string_lst: list[str], str_format: str) -> list[datetime]:
    return [datetime.strptime(date_str, str_format) for date_str in string_lst]

In [ ]:
print(column_names)

In [ ]:
transaction_date = [row['transaction_date'] for row in data]
print(transaction_date)
purchase_lb = [row['purchase_lb'] for row in data]
purchase_ub = [row['purchase_ub'] for row in data]
sale_lb = [-row['sale_lb'] for row in data]
sale_ub = [-row['sale_ub'] for row in data]
transaction_date = convert_date_string_lst_to_datetime(transaction_date, '%d-%m-%Y')
print(transaction_date)

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
from plotly.utils import PlotlyJSONEncoder
import pandas
from pandas import DataFrame

In [ ]:
purchase_sales_lb_ub_df = DataFrame([transaction_date, purchase_lb, purchase_ub, sale_lb, sale_ub]).transpose()
purchase_sales_lb_ub_df.columns = ['transaction_date', 'purchase_lb', 'purchase_ub', 'sale_lb', 'sale_ub']

In [ ]:
print(purchase_sales_lb_ub_df.columns)

In [ ]:

purchase_lb_graph = go.Histogram(x=purchase_sales_lb_ub_df['transaction_date'],
                                 y=purchase_sales_lb_ub_df['purchase_lb'],
                                 name='purchase_lb',
                                 texttemplate="%{x}", textfont_size=20)
purchase_ub_graph = go.Histogram(x=purchase_sales_lb_ub_df['transaction_date'],
                                 y=purchase_sales_lb_ub_df['purchase_ub'],
                                 name='purchase_ub',
                                 texttemplate="%{x}", textfont_size=20)
sale_lb_graph = go.Histogram(x=purchase_sales_lb_ub_df['transaction_date'],
                             y=purchase_sales_lb_ub_df['sale_lb'],
                             name='sale_lb',
                             texttemplate="%{x}", textfont_size=20)
sale_ub_graph = go.Histogram(x=purchase_sales_lb_ub_df['transaction_date'],
                             y=purchase_sales_lb_ub_df['sale_ub'],
                             name='sale_ub',
                             texttemplate="%{x}", textfont_size=20)

In [ ]:
fig = go.Figure()
fig.add_trace(purchase_ub_graph)
fig.add_trace(purchase_lb_graph)
fig.add_trace(sale_ub_graph)
fig.add_trace(sale_lb_graph)
fig.update_layout(barmode='group')
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
fig_alt = px.line(purchase_sales_lb_ub_df, x = 'transaction_date', y = ['purchase_lb', 'purchase_ub', 'sale_lb', 'sale_ub'])
fig_alt.update_traces(mode='markers+lines')
fig_alt.show()


In [ ]:
fig_third = px.histogram(purchase_sales_lb_ub_df, x = 'transaction_date', y=purchase_sales_lb_ub_df.columns, barmode='overlay')
fig_third.show()

In [ ]:
print(type(fig_third))

In [ ]:
import json

In [ ]:
graphJSON = json.dumps(fig_third, cls=PlotlyJSONEncoder)

In [ ]:
print(type(graphJSON))